https://deap.readthedocs.io/en/master/examples/es_fctmin.html

In [1]:
import array
import random

import numpy as np

from deap import algorithms
from deap import base
from deap import benchmarks
from deap import creator
from deap import tools

C:\Program Files\Anaconda3\lib\site-packages\deap\tools\_hypervolume\pyhv.py:33: ImportWarning: Falling back to the python version of hypervolume module. Expect this to be very slow.
  "module. Expect this to be very slow.", ImportWarning)


In [3]:
IND_SIZE = 30
MIN_VALUE = 4
MAX_VALUE = 5
MIN_STRATEGY = 0.5
MAX_STRATEGY = 3

In [4]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", array.array, typecode="d", fitness=creator.FitnessMin,
              strategy=None)
creator.create("Strategy", array.array, typecode="d")


In [7]:
def generateES(icls, scls, size, imin, imax, smin, smax):
    """Individual generator"""
    ind = icls(random.uniform(imin, imax) for _ in range(size))
    ind.strategy = scls(random.uniform(smin, smax) for _ in range(size))
    return ind

def checkStrategy(minstrategy):
    def decorator(func):
        def wrapper(*args, **kargs):
            children = func(*args, **kargs)
            for child in children:
                for i, s in enumerate(child.strategy):
                    if s < minstrategy:
                        child.strategy[i] = minstrategy
            return children
        return wrapper
    return decorator

toolbox = base.Toolbox()
toolbox.register("individual", generateES, creator.Individual, creator.Strategy,
                IND_SIZE, MIN_VALUE, MAX_VALUE, MIN_STRATEGY, MAX_STRATEGY)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("mate", tools.cxESBlend, alpha=0.1)
toolbox.register("mutate", tools.mutESLogNormal, c=1.0, indpb=0.03)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", benchmarks.sphere)

toolbox.decorate("mate", checkStrategy(MIN_STRATEGY))
toolbox.decorate("mutate", checkStrategy(MIN_STRATEGY))

In [9]:
def main():
    random.seed()
    MU, LAMBDA = 10, 100
    pop = toolbox.population(n=MU)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)
    
    pop, logbook = algorithms.eaMuCommaLambda(pop, toolbox, mu=MU, lambda_=LAMBDA, 
        cxpb=0.6, mutpb=0.3, ngen=500, stats=stats, halloffame=hof)
    
    return pop, logbook, hof
    
if __name__ == "__main__":
    main()

gen	nevals	avg    	std    	min    	max    
0  	10    	604.259	12.3592	586.928	626.697
1  	92    	589.89 	6.3105 	574.769	597.321
2  	89    	581.2  	6.41707	570.9  	589.6  
3  	91    	576.636	3.68932	569.199	583.326
4  	93    	571.955	4.21417	564.545	581.369
5  	87    	567.149	3.76962	557.78 	571.61 
6  	91    	565.651	4.45002	554.055	570.662
7  	89    	562.685	5.09558	551.274	567.391
8  	94    	554.408	9.47204	536.623	565.858
9  	88    	543.615	9.43684	523.91 	556.325
10 	93    	535.151	8.32928	521.817	548.514
11 	92    	532.271	5.24664	524.365	539.073
12 	85    	524.94 	8.50292	501.099	533.674
13 	88    	516.455	9.49508	500.237	528.727
14 	97    	505.246	8.42469	492.357	519.547
15 	96    	492.277	7.08602	478.73 	500.406
16 	91    	485.402	7.53938	473.392	494.537
17 	85    	474.138	5.04693	463.866	483.942
18 	90    	466.36 	4.76961	459.583	475.46 
19 	87    	459.803	4.49746	452.886	467.012
20 	92    	453.055	10.4797	424.125	463.308
21 	89    	446.088	8.11363	432.675	455.242
22 	91    	

187	92    	13.524 	0.227585 	13.2177	13.8787
188	86    	13.4329	0.0864494	13.3   	13.6518
189	91    	12.4358	1.87106  	8.69533	13.4569
190	87    	10.9593	2.07545  	8.69533	13.3793
191	90    	9.62708	1.19611  	8.68654	12.8243
192	88    	8.9626 	0.325813 	8.68654	9.64122
193	92    	8.80906	0.220271 	8.64016	9.23797
194	91    	8.65367	0.0259095	8.59555	8.6887 
195	83    	8.62721	0.026716 	8.58625	8.65844
196	89    	8.58529	0.0402839	8.47673	8.63892
197	94    	8.5655 	0.0388318	8.47673	8.59816
198	87    	8.50753	0.099482 	8.25236	8.58627
199	89    	8.46251	0.0875578	8.27071	8.56559
200	89    	8.35553	0.151445 	7.97545	8.52652
201	95    	8.10757	0.246472 	7.79051	8.39711
202	89    	7.89376	0.184921 	7.63002	8.3526 
203	91    	7.74066	0.0845714	7.59395	7.85884
204	77    	7.55546	0.179748 	7.28042	7.82857
205	87    	7.39917	0.0786743	7.30373	7.5316 
206	91    	7.21784	0.126984 	7.10205	7.5316 
207	85    	7.07189	0.261972 	6.3096 	7.24792
208	91    	6.95353	0.326498 	6.2786 	7.24252
209	91    

373	89    	0.48815 	0.00585248 	0.477826	0.492091
374	95    	0.481904	0.00445085 	0.477826	0.489646
375	92    	0.475662	0.0083409  	0.460192	0.488487
376	87    	0.466299	0.0068714  	0.460192	0.477826
377	90    	0.461537	0.00142935 	0.460192	0.465024
378	92    	0.46083 	0.000621549	0.460115	0.461959
379	88    	0.460519	0.000406328	0.460145	0.461321
380	91    	0.460361	0.000319217	0.460146	0.461269
381	88    	0.460214	6.31479e-05	0.460146	0.460337
382	91    	0.460193	3.87928e-05	0.460148	0.460252
383	88    	0.460174	2.84147e-05	0.460148	0.460235
384	86    	0.460162	1.37953e-05	0.460147	0.460191
385	89    	0.460152	5.77258e-06	0.460145	0.460163
386	88    	0.460147	2.54803e-06	0.460145	0.460154
387	88    	0.460146	1.24416e-06	0.460145	0.46015 
388	92    	0.460146	1.34228e-06	0.460145	0.46015 
389	92    	0.460145	1.48876e-07	0.460145	0.460146
390	88    	0.460141	1.32897e-05	0.460101	0.460145
391	92    	0.459841	0.000258216	0.45952 	0.460145
392	94    	0.459633	0.000149703	0.45952 	0.459994
